[This notebook showing example of toy GO terms]

Rank all GO biological processes by the similarity with the LLM term. 

* % of other GO names have smaller semantic similarity with the GPT-4 name comparing to the assgined GO name



In [1]:
import pandas as pd
all_go = pd.read_csv('data/go_terms.csv', index_col=0)
print(all_go.shape)


(11943, 4)


## Step 1 get the word embeddings for all the go terms (only need to run once for all)

In [2]:
## create embeddings for all GO Terms and save the embeddings 
from semanticSimFunctions import getSentenceEmbedding
from transformers import AutoTokenizer, AutoModel
import pandas as pd

SapBERT_tokenizer = AutoTokenizer.from_pretrained('cambridgeltl/SapBERT-from-PubMedBERT-fulltext')
SapBERT_model = AutoModel.from_pretrained('cambridgeltl/SapBERT-from-PubMedBERT-fulltext')

all_go = pd.read_csv('data/go_terms.csv', index_col=0)

all_go_terms = all_go['Term_Description'].tolist()

all_go_terms_embeddings_dict = {}
for i, go_term in enumerate(all_go_terms):
    tensor = getSentenceEmbedding(go_term, SapBERT_tokenizer, SapBERT_model)
    all_go_terms_embeddings_dict[go_term] = tensor.numpy()  # Convert to numpy array

import pickle
with open('data/all_go_terms_embeddings_dict.pkl', 'wb') as handle:  
    pickle.dump(all_go_terms_embeddings_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [3]:
## create embeddings for all GO Terms and save the embeddings 
from semanticSimFunctions import getSentenceEmbedding
from transformers import AutoTokenizer, AutoModel
import pandas as pd

SapBERT_tokenizer = AutoTokenizer.from_pretrained('cambridgeltl/SapBERT-from-PubMedBERT-fulltext')
SapBERT_model = AutoModel.from_pretrained('cambridgeltl/SapBERT-from-PubMedBERT-fulltext')

for branch in ['CC', 'MF']:
    
    all_go = pd.read_csv(f'data/GO_term_analysis/CC_MF_branch/{branch}_go_terms.csv', index_col=0)

    all_go_terms = all_go['Term_Description'].tolist()

    all_go_terms_embeddings_dict = {}
    for i, go_term in enumerate(all_go_terms):
        tensor = getSentenceEmbedding(go_term, SapBERT_tokenizer, SapBERT_model)
        all_go_terms_embeddings_dict[go_term] = tensor.numpy()  # Convert to numpy array

    import pickle
    with open(f'data/GO_term_analysis/CC_MF_branch/{branch}_go_terms_embeddings_dict.pkl', 'wb') as handle:  
        pickle.dump(all_go_terms_embeddings_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [4]:
#check if embeddings are saved correctly
import pickle
with open('data/all_go_terms_embeddings_dict.pkl', 'rb') as handle:
    all_go_terms_embeddings_dict = pickle.load(handle)
print(len(all_go_terms_embeddings_dict))
# all_go_terms_embeddings_dict['cellular response to DNA damage stimulus']

11943


In [5]:
#check if embeddings are saved correctly
import pickle
for branch in ['CC', 'MF']:
    with open(f'data/GO_term_analysis/CC_MF_branch/{branch}_go_terms_embeddings_dict.pkl', 'rb') as handle:
        all_go_terms_embeddings_dict = pickle.load(handle)
    print(len(all_go_terms_embeddings_dict))
    

1677
3399


## Step2: iterate through each GO term and its corresponsing LLM term, rank the similarity score of the LLM with all GO terms and find where is the trueGO-LLM term is among the list


When running for the 1000 gene set, used the python function rank_GOterm_LLM_sim.py to run at the background

 ```
 python rank_GOterm_LLM_sim_rand.py --input_file data/GO_term_analysis/LLM_processed_selected_1000_go_terms.tsv --emb_file data/all_go_terms_3to100_embeddings_dict.pkl --topn 50 --output_file data/GO_term_analysis/simrank_LLM_processed_selected_1000_go_terms.tsv --background_file data/GO_term_analysis/all_go_sim_scores.txt
 ```
the code at the bottom is just an example

In [ ]:
%run rank_GOterm_LLM_sim_rand.py --input_file data/GO_term_analysis/model_compare/LLM_processed_model_compare_100set_gpt_4.tsv --emb_file data/all_go_terms_3to100_embeddings_dict.pkl --topn 3 --output_file data/GO_term_analysis/model_compare/sim_rank_LLM_processed_model_compare_100set_gpt_4.tsv --background_file data/GO_term_analysis/all_go_sim_scores_3to100_gpt4.txt

In [1]:
from glob import glob
files = glob('data/GO_term_analysis/model_compare/LLM_processed_model_compare_100set*.tsv')
# print(files)
for file in files:
    model = '_'.join(file.split('/')[-1].split('.')[0].split('_')[-2:])
    print(model)
    if model == 'gpt_4':
        continue
    else:
        print(f'python rank_GOterm_LLM_sim_rand.py --input_file {file} --emb_file data/all_go_terms_3to100_embeddings_dict.pkl --topn 3 --output_file data/GO_term_analysis/model_compare/sim_rank_LLM_processed_model_compare_100set_{model}.tsv')
    

gpt_35
python rank_GOterm_LLM_sim_rand.py --input_file data/GO_term_analysis/model_compare/LLM_processed_model_compare_100set_gpt_35.tsv --emb_file data/all_go_terms_3to100_embeddings_dict.pkl --topn 3 --output_file data/GO_term_analysis/model_compare/sim_rank_LLM_processed_model_compare_100set_gpt_35.tsv
mixtral_latest
python rank_GOterm_LLM_sim_rand.py --input_file data/GO_term_analysis/model_compare/LLM_processed_model_compare_100set_mixtral_latest.tsv --emb_file data/all_go_terms_3to100_embeddings_dict.pkl --topn 3 --output_file data/GO_term_analysis/model_compare/sim_rank_LLM_processed_model_compare_100set_mixtral_latest.tsv
gpt_4
llama2_70b
python rank_GOterm_LLM_sim_rand.py --input_file data/GO_term_analysis/model_compare/LLM_processed_model_compare_100set_llama2_70b.tsv --emb_file data/all_go_terms_3to100_embeddings_dict.pkl --topn 3 --output_file data/GO_term_analysis/model_compare/sim_rank_LLM_processed_model_compare_100set_llama2_70b.tsv
mixtral_instruct
python rank_GOterm_L

In [1]:
from glob import glob

# print(files)
for branch in ['CC', 'MF']:
    file = glob(f'data/GO_term_analysis/CC_MF_branch/LLM_processed_selected_1000*{branch}terms.tsv')
    
    print(f'python rank_GOterm_LLM_sim_rand_new.py --input_file {file[0]} --emb_file data/GO_term_analysis/CC_MF_branch/{branch}_go_terms_embeddings_dict.pkl --topn 3 --output_file data/GO_term_analysis/CC_MF_branch/sim_rank_LLM_processed_selected_1000_go_{branch}terms.tsv --background_file data/GO_term_analysis/CC_MF_branch/{branch}_go_sim_scores.txt')
    

python rank_GOterm_LLM_sim_rand_new.py --input_file data/GO_term_analysis/CC_MF_branch/LLM_processed_selected_1000_go_CCterms.tsv --emb_file data/GO_term_analysis/CC_MF_branch/CC_go_terms_embeddings_dict.pkl --topn 3 --output_file data/GO_term_analysis/CC_MF_branch/sim_rank_LLM_processed_selected_1000_go_CCterms.tsv --background_file data/GO_term_analysis/CC_MF_branch/CC_go_sim_scores.txt
python rank_GOterm_LLM_sim_rand_new.py --input_file data/GO_term_analysis/CC_MF_branch/LLM_processed_selected_1000_go_MFterms.tsv --emb_file data/GO_term_analysis/CC_MF_branch/MF_go_terms_embeddings_dict.pkl --topn 3 --output_file data/GO_term_analysis/CC_MF_branch/sim_rank_LLM_processed_selected_1000_go_MFterms.tsv --background_file data/GO_term_analysis/CC_MF_branch/MF_go_sim_scores.txt


In [ ]:
%run rank_GOterm_LLM_sim_rand_new.py --input_file data/GO_term_analysis/LLM_processed_selected_1000_go_terms.tsv --emb_file data/all_go_terms_embeddings_dict.pkl --topn 3 --output_file data/GO_term_analysis/simrank_LLM_processed_selected_1000_go_terms.tsv --background_file data/GO_term_analysis/all_go_sim_scores_gpt4.txt

In [6]:
# sanity check
df = pd.read_csv('data/GO_term_analysis/simrank_LLM_processed_toy_example.tsv', sep='\t', index_col=0)
df.head()

,Genes,Gene_Count,Term_Description,LLM Name,LLM Analysis,LLM_name_GO_term_sim,sim_rank,true_GO_term_sim_percentile,random_GO_name,random_go_llm_sim,random_sim_rank,random_sim_percentile,top_50_hits,top_50_sim
GO,,,,,,,,,,,,,,
GO:0032385,LDLRAP1 SCP2D1 ANXA2 SCP2,4,positive regulation of intracellular cholester...,Lipid Transport and Metabolism,"Proteins: LDLRAP1, SCP2D1, ANXA2, SCP2\n\n1. L...",0.382285,1673,0.863026,Tie signaling pathway,0.344493,2832,0.768135,lipid metabolic process|lipid transport|cellul...,0.8934768|0.8777427|0.8366494|0.82344246|0.800...
GO:0002468,NOD1 HLA-DRA CLEC4A HLA-DRB1 CCL21 NOD2 CCL19 ...,15,dendritic cell antigen processing and presenta...,Antigen Presentation and Immune Response Modul...,The primary biological process performed by th...,0.725125,12,0.999018,positive regulation of reciprocal meiotic reco...,0.352192,3034,0.751597,regulation of antigen processing and presentat...,0.8205818|0.788555|0.77849185|0.7683631|0.7679...
GO:0033683,OGG1 ERCC5 XPA ERCC4 NTHL1,5,"nucleotide-excision repair, DNA incision",DNA Repair,"The system of interacting proteins OGG1, ERCC5...",0.688920,26,0.997871,Tie signaling pathway,0.342632,3393,0.722204,DNA repair|DNA synthesis involved in DNA repai...,0.9999999|0.8868111|0.80208004|0.7669432|0.763...
GO:0035672,SLC7A11 SLC25A39 SLC26A6 ABCB9 SLC15A4 ABCC5 C...,15,oligopeptide transmembrane transport,Ion and Nutrient Transport Regulation,The primary biological process performed by th...,0.495159,639,0.947683,hypomethylation of CpG island,0.294337,7315,0.401097,regulation of ion transport|regulation of ion ...,0.84507704|0.797101|0.77553225|0.7660217|0.749...
GO:0048023,OPN3 CDH3 ATP7A APPL1 ASIP RAB38 ZEB2 TYRP1 GIPC1,9,positive regulation of melanin biosynthetic pr...,Melanogenesis Regulation,"Proteins: OPN3, CDH3, ATP7A, APPL1, ASIP, RAB3...",0.637935,29,0.997626,RNA (guanine-N7)-methylation,0.255608,9349,0.234567,regulation of melanocyte differentiation|regul...,0.8627623|0.8499877|0.82896507|0.80347717|0.79...


### Check the rank similarity result of the 1000 gene sets 

In [2]:
import pandas as pd

rank_sim_df = pd.read_csv('data/GO_term_analysis/simrank_LLM_processed_selected_1000_go_terms.tsv', sep='\t')
## if duplicate
print(sum(rank_sim_df.duplicated(subset=['GO'])))
print(sum(rank_sim_df.duplicated(subset=['gpt_4_default Analysis'])))

#if nan

print(sum(rank_sim_df['true_GO_term_sim_percentile'].isna()))

# check how many gpt4 names are 'System of unrelated proteins'
print(sum(rank_sim_df['gpt_4_default Name'] == 'System of unrelated proteins'))
# for each of them set the semantic similarity to 0, rank to max number of GO terms, and percentile to 0
rank_sim_df.loc[rank_sim_df['gpt_4_default Name'] == 'System of unrelated proteins', 'LLM_name_GO_term_sim'] = 0
rank_sim_df.loc[rank_sim_df['gpt_4_default Name'] == 'System of unrelated proteins', 'sim_rank'] = 11943
rank_sim_df.loc[rank_sim_df['gpt_4_default Name'] == 'System of unrelated proteins', 'true_GO_term_sim_percentile'] = 0

print(sum(rank_sim_df['true_GO_term_sim_percentile']==0))
rank_sim_df.to_csv('data/GO_term_analysis/simrank_LLM_processed_selected_1000_go_terms.tsv', sep='\t', index=False)


0
0
0
26
26


In [8]:
# change column order and remove the contaminated genes columns
rank_sim_df = rank_sim_df[['GO', 'Genes', 'Gene_Count', 'Term_Description',
       'gpt_4_default Name', 'gpt_4_default Analysis', 'gpt_4_default Score', 'gpt_4_default Score Bin',
       'LLM_name_GO_term_sim', 'sim_rank', 'true_GO_term_sim_percentile',
       'random_GO_name', 'random_go_llm_sim', 'random_sim_rank',
       'random_sim_percentile', 'top_3_hits', 'top_3_sim']]
rank_sim_df.columns

Index(['GO', 'Genes', 'Gene_Count', 'Term_Description', 'gpt_4_default Name',
       'gpt_4_default Analysis', 'gpt_4_default Score',
       'gpt_4_default Score Bin', 'LLM_name_GO_term_sim', 'sim_rank',
       'true_GO_term_sim_percentile', 'random_GO_name', 'random_go_llm_sim',
       'random_sim_rank', 'random_sim_percentile', 'top_3_hits', 'top_3_sim'],
      dtype='object')

In [7]:
rank_sim_df.to_csv('data/GO_term_analysis/simrank_LLM_processed_selected_1000_go_terms.tsv', sep='\t', index=False)

In [24]:

## half point of the similarity distribution
rank_sim_sorted = rank_sim_df.sort_values(by='true_GO_term_sim_percentile', ascending=False)
print('half of the sample have the percentile score higher than: ',rank_sim_sorted.iloc[500-1]['true_GO_term_sim_percentile'])

## number of GO terms in top 10% of similarities
print('number of GO terms in top 10%: ', sum(1-rank_sim_df['true_GO_term_sim_percentile'] <= 0.1))
print('number of GO terms in bottom 10%: ', sum(1-rank_sim_df['true_GO_term_sim_percentile'] >= 0.9))

## number of GO terms ranked top 10 of similarities

print('number of GO terms ranked top 10: ', sum(rank_sim_df['sim_rank'] <= 10))

half of the sample have the percentile score higher than:  0.9794858913170896
number of GO terms in top 10%:  685
number of GO terms in bottom 10%:  41
number of GO terms ranked top 10:  123


In [25]:
# create a table for ones that output 'System of unrelated proteins'

unnamed_sets = rank_sim_df.loc[rank_sim_df['gpt_4_default Name'] == 'System of unrelated proteins']
print(len(unnamed_sets)) ## 26 out of the 1000 GO terms are unnamed
# check the size 
print(unnamed_sets['Gene_Count'].describe()) # the unnamed sets are not dependent on the size 

unnamed_sets.to_csv('data/GO_term_analysis/sets_no_gpt4name.tsv', sep='\t', index=False)

26
count    26.000000
mean     23.500000
std      25.842987
min       3.000000
25%       5.250000
50%      11.000000
75%      32.750000
max      84.000000
Name: Gene_Count, dtype: float64


In [29]:
# filter out the unnamed sets
rank_sim_df_filter = rank_sim_df.loc[rank_sim_df['gpt_4_default Name'] != 'System of unrelated proteins']
# check anything with score ==0
print(sum(rank_sim_df_filter['gpt_4_default Score'] == 0))

1


In [6]:
# rank the GO terms by the similarity of LLM name and GO term and pick top 25 and bottom 25 for manual evaluation
rank_sim_df.sort_values(by=['LLM_name_GO_term_sim'], ascending=False, inplace=True)
top = rank_sim_df.head(25)
bottom = rank_sim_df.tail(25)
combine_df = pd.concat([top,bottom], ignore_index=True)


# # add a column to randomly assign number from 1-5, each has the same number of GO terms
# team = [1,2,3,4,5]*10
# import random
# random.seed(2023)
# random.shuffle(team)
# combine_df['team'] = team



combine_df.to_csv('data/GO_term_analysis/best_25_worst_25_similarity_among1000GO.tsv', sep='\t', index=False)


### MF and CC branch similarity

In [1]:
import pandas as pd

for branch in ['CC', 'MF']:
    print(branch)
    all_go = pd.read_csv(f'data/GO_term_analysis/CC_MF_branch/{branch}_go_terms.csv', index_col=0)
    # filtered go terms to size range 3-100 
    # since we picked term size from 3-100, it makes more sense to filter the go term pool to that size range
    min = 3
    max = 100
    filtered_go = all_go[(all_go['Gene_Count'] >= min) & (all_go['Gene_Count'] <= max)]
    max_rank = filtered_go.shape[0]
    
    rank_sim_df = pd.read_csv(f'data/GO_term_analysis/CC_MF_branch/sim_rank_LLM_processed_selected_1000_go_{branch}terms.tsv', sep='\t')
    ## if duplicate
    print(sum(rank_sim_df.duplicated(subset=['GO'])))
    print(sum(rank_sim_df.duplicated(subset=['gpt_4_default Analysis'])))

    #if nan

    print(sum(rank_sim_df['true_GO_term_sim_percentile'].isna()))

    # check how many gpt4 names are 'System of unrelated proteins'
    print(sum(rank_sim_df['gpt_4_default Name'] == 'System of unrelated proteins'))
    print(sum(rank_sim_df['gpt_4_default Score'] == 0))
    # for each of them set the semantic similarity to 0, rank to max number of GO terms, and percentile to 1
    rank_sim_df.loc[rank_sim_df['gpt_4_default Name'] == 'System of unrelated proteins', 'LLM_name_GO_term_sim'] = 0
    rank_sim_df.loc[rank_sim_df['gpt_4_default Name'] == 'System of unrelated proteins', 'sim_rank'] = max_rank
    rank_sim_df.loc[rank_sim_df['gpt_4_default Name'] == 'System of unrelated proteins', 'true_GO_term_sim_percentile'] = 0

    print(sum(rank_sim_df['true_GO_term_sim_percentile']==0))

    rank_sim_df.to_csv(f'data/GO_term_analysis/CC_MF_branch/sim_rank_LLM_processed_selected_1000_go_{branch}terms.tsv', sep='\t', index=False)


CC
0
0
0
17
17
17
MF
0
0
0
8
8
8
